In [1]:
##### Copyright 2020 Google LLC
#
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introduction

In this notebook, we show how to quantize a model using AutoQKeras.

As usual, let's first make sure we are using Python 3.

In [2]:
import sys
print(sys.version)

3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]


Now, let's load some packages we will need to run AutoQKeras.

In [3]:
import warnings
warnings.filterwarnings("ignore")

import json
import pprint
import numpy as np
import six
import tempfile
import tensorflow.compat.v2 as tf
# V2 Behavior is necessary to use TF2 APIs before TF2 is default TF version internally.
tf.enable_v2_behavior()
from tensorflow.keras.optimizers import *

from qkeras.autoqkeras import *
from qkeras import *
from qkeras.utils import model_quantize
from qkeras.qtools import run_qtools
from qkeras.qtools import settings as qtools_settings

from tensorflow.keras.utils import to_categorical
import tensorflow_datasets as tfds

print("using tensorflow", tf.__version__)

2022-06-21 11:50:29.402097: I tensorflow/core/util/util.cc:168] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-06-21 11:50:29.409510: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-21 11:50:29.409647: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


using tensorflow 2.10.0-dev20220407


Let's define `get_data` and `get_model` as you may not have stand alone access to examples directory inside autoqkeras.

In [2]:
def get_data(dataset_name, fast=False):
  """Returns dataset from tfds."""
  ds_train = tfds.load(name=dataset_name, split="train", batch_size=-1)
  ds_test = tfds.load(name=dataset_name, split="test", batch_size=-1)

  dataset = tfds.as_numpy(ds_train)
  x_train, y_train = dataset["image"].astype(np.float32), dataset["label"]

  dataset = tfds.as_numpy(ds_test)
  x_test, y_test = dataset["image"].astype(np.float32), dataset["label"]

  if len(x_train.shape) == 3:
    x_train = x_train.reshape(x_train.shape + (1,))
    x_test = x_test.reshape(x_test.shape + (1,))

  x_train /= 256.0
  x_test /= 256.0

  x_mean = np.mean(x_train, axis=0)

  x_train -= x_mean
  x_test -= x_mean

  nb_classes = np.max(y_train) + 1
  y_train = to_categorical(y_train, nb_classes)
  y_test = to_categorical(y_test, nb_classes)

  print(x_train.shape[0], "train samples")
  print(x_test.shape[0], "test samples")
  return (x_train, y_train), (x_test, y_test)


In [3]:
from tensorflow.keras.initializers import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *

boosted_model =  tf.keras.models.load_model('../pb_file')

boosted_model.summary()

2022-06-23 16:13:53.616478: I tensorflow/core/util/util.cc:168] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-06-23 16:13:53.622138: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-23 16:13:53.622155: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


NameError: name 'tf' is not defined

`AutoQKeras` has some examples on how to run with `mnist`, `fashion_mnist`, `cifar10` and `cifar100`.

In [7]:
DATASET = "mnist"
(x_train, y_train), (x_test, y_test) = get_data(DATASET)

2022-06-21 11:50:59.532103: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

...

2022-06-21 11:51:01.894280: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-21 11:51:01.894318: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-21 11:51:01.894345: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (martop): /proc/driver/nvidia/version does not exist
2022-06-21 11:51:01.894830: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Shuffling /atlas/bonnet/tensorflow_datasets/mnist/3.0.1.incompleteDRUTR4/mnist-train.tfrecord*...:   0%|      …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /atlas/bonnet/tensorflow_datasets/mnist/3.0.1.incompleteDRUTR4/mnist-test.tfrecord*...:   0%|       …

Dataset mnist downloaded and prepared to /atlas/bonnet/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
60000 train samples
10000 test samples


Before we create the model, let's see if we can perform distributed training.

In [8]:
physical_devices = tf.config.list_physical_devices()
for d in physical_devices:
  print(d)

PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')


In [9]:
has_tpus = np.any([d.device_type == "TPU" for d in physical_devices])

if has_tpus:
  TPU_WORKER = 'local'

  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
      tpu=TPU_WORKER, job_name='tpu_worker')
  if TPU_WORKER != 'local':
    tf.config.experimental_connect_to_cluster(resolver, protocol='grpc+loas')
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
  print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

  cur_strategy = strategy
else:
  cur_strategy = tf.distribute.get_strategy()

Now we can create the model with the distributed strategy in place if TPUs are available. We have some test models that we can use, or you can build your own models. 

In [10]:
with cur_strategy.scope():
  model = get_model(DATASET)
  custom_objects = {}

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_0 (Conv2D)           (None, 14, 14, 16)        144       
                                                                 
 bn_0 (BatchNormalization)   (None, 14, 14, 16)        64        
                                                                 
 act_0 (Activation)          (None, 14, 14, 16)        0         
                                                                 
 drop_0 (Dropout)            (None, 14, 14, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 32)          4608      
                                                                 
 bn_1 (BatchNormalization)   (None, 7, 7, 32)          128   

Let's see the accuracy on a unquantized model.

In [11]:
with cur_strategy.scope():
  optimizer = Adam(lr=0.02)
  model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["acc"])
  model.fit(x_train, y_train, epochs=10, batch_size=2048, steps_per_epoch=29, validation_data=(x_test, y_test))

Epoch 1/10
29/29 [==============================] - 6s 111ms/step - loss: 0.5309 - acc: 0.8321 - val_loss: 1.1922 - val_acc: 0.8741
Epoch 2/10
29/29 [==============================] - 3s 91ms/step - loss: 0.1657 - acc: 0.9485 - val_loss: 0.2747 - val_acc: 0.9313
Epoch 3/10
29/29 [==============================] - 3s 100ms/step - loss: 0.1230 - acc: 0.9611 - val_loss: 0.1031 - val_acc: 0.9681
Epoch 4/10
29/29 [==============================] - 3s 107ms/step - loss: 0.1030 - acc: 0.9676 - val_loss: 0.1136 - val_acc: 0.9630
Epoch 5/10
29/29 [==============================] - 3s 93ms/step - loss: 0.0883 - acc: 0.9719 - val_loss: 0.0648 - val_acc: 0.9786
Epoch 6/10
29/29 [==============================] - 3s 89ms/step - loss: 0.0842 - acc: 0.9741 - val_loss: 0.0657 - val_acc: 0.9788
Epoch 7/10
29/29 [==============================] - 3s 91ms/step - loss: 0.0745 - acc: 0.9770 - val_loss: 0.0443 - val_acc: 0.9861
Epoch 8/10
29/29 [==============================] - 3s 104ms/step - loss: 0.0726

For `mnist`, we should get 99% validation accuracy, and for `fashion_mnist`, we should get around 86% of validation accuracy. Let's get a metric for high-level estimation of energy of this model. 



In [12]:
  reference_internal = "fp32"
  reference_accumulator = "fp32"

  q = run_qtools.QTools(
      model,
      # energy calculation using a given process
      # "horowitz" refers to 45nm process published at
      # M. Horowitz, "1.1 Computing's energy problem (and what we can do about
      # it), "2014 IEEE International Solid-State Circuits Conference Digest of
      # Technical Papers (ISSCC), San Francisco, CA, 2014, pp. 10-14, 
      # doi: 10.1109/ISSCC.2014.6757323.
      process="horowitz",
      # quantizers for model input
      source_quantizers=[quantized_bits(8, 0, 1)],
      is_inference=False,
      # absolute path (including filename) of the model weights
      # in the future, we will attempt to optimize the power model
      # by using weight information, although it can be used to further
      # optimize QBatchNormalization.
      weights_path=None,
      # keras_quantizer to quantize weight/bias in un-quantized keras layers
      keras_quantizer=reference_internal,
      # keras_quantizer to quantize MAC in un-quantized keras layers
      keras_accumulator=reference_accumulator,
      # whether calculate baseline energy
      for_reference=True)
  
# caculate energy of the derived data type map.
energy_dict = q.pe(
    # whether to store parameters in dram, sram, or fixed
    weights_on_memory="sram",
    # store activations in dram or sram
    activations_on_memory="sram",
    # minimum sram size in number of bits. Let's assume a 16MB SRAM.
    min_sram_size=8*16*1024*1024,
    # whether load data from dram to sram (consider sram as a cache
    # for dram. If false, we will assume data will be already in SRAM
    rd_wr_on_io=False)

# get stats of energy distribution in each layer
energy_profile = q.extract_energy_profile(
    qtools_settings.cfg.include_energy, energy_dict)
# extract sum of energy of each layer according to the rule specified in
# qtools_settings.cfg.include_energy
total_energy = q.extract_energy_sum(
    qtools_settings.cfg.include_energy, energy_dict)

pprint.pprint(energy_profile)
print()
print("Total energy: {:.2f} uJ".format(total_energy / 1000000.0))

{'act_0': {'energy': {'inputs': 5964.34,
                      'op_cost': 0.0,
                      'outputs': 5964.34,
                      'parameters': 0.0},
           'total': 5964.34},
 'act_1': {'energy': {'inputs': 2982.17,
                      'op_cost': 0.0,
                      'outputs': 2982.17,
                      'parameters': 0.0},
           'total': 2982.17},
 'act_2': {'energy': {'inputs': 1460.66,
                      'op_cost': 0.0,
                      'outputs': 1460.66,
                      'parameters': 0.0},
           'total': 1460.66},
 'act_3': {'energy': {'inputs': 486.89,
                      'op_cost': 0.0,
                      'outputs': 486.89,
                      'parameters': 0.0},
           'total': 486.89},
 'act_4': {'energy': {'inputs': 243.44,
                      'op_cost': 0.0,
                      'outputs': 243.44,
                      'parameters': 0.0},
           'total': 243.44},
 'bn_0': {'energy': {'inputs': 5964.34,
 

During the computation, we had a dictionary that outlines the energy per layer (`energy_profile`), and total energy (`total_energy`). The reader should remember that `energy_profile` may need additional filtering as implementations will fuse some
layers. When we compute the `total_energy`, we consider an approximation that some layers will be fused to compute the final energy number. For example, a convolution layer followed by an activation layer will be fused into a single layer so that the output of the convolution layer is not used.

You have to remember that our high-level model for energy has several assumptions:

The energy of a layer is estimated as `energy(layer) = energy(input) + energy(parameters) + energy(MAC) + energy(output)`.

1) Reading inputs, parameters and outputs consider only _compulsory_ accesses, i.e. first access to the data, which is independent of the hardware architecture. If you remember _The 3 C's of Caches_ (https://courses.cs.washington.edu/courses/cse410/99au/lectures/Lecture-10-18/tsld035.htm) other types of accesses will depend on the accelerator architecture.

2) For the multiply-and-add (MAC) energy estimation, we only consider the energy to compute the MAC, but not any other type energy. For example, in a real accelerator, you have registers, glue logic, pipeline logic that will affect the overall energy profile of the device.

Although this model is simple and provides an initial estimate on what to expect, it has high-variance with respect to actual energy numbers you will find in practice, especially with respect to different architectural implementations.

We assume that the real energy `Energy(layer)` is a linear combination of the high-level energy model, i.e.`Energy(layer) = k1 * energy(layer) + k2`, where `k1` and `k2` are constants that depend on the architecture of the accelerator. One can think of `k1` as the factor that accounts for the additional storage to keep the model running, and `k2` as the additional always on logic that is required to perform the operations. If we compare the energy of two implementations with different quantizations of the same layer, let's say `layer1` and `layer2`, `Energy(layer1) > Energy(layer2)` holds true iff `energy(layer1) > energy(layer2)` for the same architecture, but for different architectures, this will not be true in general.

Despite its limitations to predict a single energy number, this model is quite good to compare the energy of two different models, or different types of quantizations, when we restrict it to a single architecture, and that's how we use it here.

# Quantizing a Model With `AutoQKeras`

To quantize this model with `AutoQKeras`, we need to define the quantization for kernels, biases and activations; forgiving factors and quantization strategy.

Below we define which quantizers are allowed for kernel, bias, activations and linear. Linear is a proxy that we use to capture `Activation("linear")` to apply quantization without applying a non-linear operation.  In some networks, we found that this trick may be necessary to better represent the quantization space.


In [13]:
quantization_config = {
        "kernel": {
                "binary": 1,
                "stochastic_binary": 1,
                "ternary": 2,
                "stochastic_ternary": 2,
                "quantized_bits(2,1,1,alpha=1.0)": 2,
                "quantized_bits(4,0,1,alpha=1.0)": 4,
                "quantized_bits(8,0,1,alpha=1.0)": 8,
                "quantized_po2(4,1)": 4
        },
        "bias": {
                "quantized_bits(4,0,1)": 4,
                "quantized_bits(8,3,1)": 8,
                "quantized_po2(4,8)": 4
        },
        "activation": {
                "binary": 1,
                "ternary": 2,
                "quantized_relu_po2(4,4)": 4,
                "quantized_relu(3,1)": 3,
                "quantized_relu(4,2)": 4,
                "quantized_relu(8,2)": 8,
                "quantized_relu(8,4)": 8,
                "quantized_relu(16,8)": 16
        },
        "linear": {
                "binary": 1,
                "ternary": 2,
                "quantized_bits(4,1)": 4,
                "quantized_bits(8,2)": 8,
                "quantized_bits(16,10)": 16
        }
}

Now let's define how to apply quantization. In the simplest form, we specify how many bits for kernels, biases and activations by layer types. Note that the entry `BatchNormalization` needs to be specified here, as we only quantize layer types specified by these patterns.  For example, a `Flatten` layer is not quantized as it does not change the data type of its inputs.

In [14]:
limit = {
    "Dense": [8, 8, 4],
    "Conv2D": [4, 8, 4],
    "DepthwiseConv2D": [4, 8, 4],
    "Activation": [4],
    "BatchNormalization": []
}

Here, we are specifying that we want to use at most 4 bits for weights and activations, and at most 8 bits for biases in convolutional and depthwise convolutions, but we allow up to 8 bits for kernels in dense layers.

Let's define now the forgiving factor. We will consider energy minimization as a goal as follows.  Here, we are saying that we allow 8% reduction in accuracy for a 2x reduction in energy, both reference and trials have parameters and activations on SRAM, both reference model and quantization trials do not read/write from DRAM on I/O operations, and we should consider both experiments to use SRAMs with minimum tensor sizes (commonly called distributed SRAM implementation).

We also need to specify the quantizers for the inputs. In this case, we want to use `int8` as source quantizers. Other possible types are `int16`, `int32`, `fp16` or `fp32`, besides `QKeras` quantizer types.

Finally, to be fair, we want to compare our quantization against fixed-point 8-bit inputs, outputs, activations, weights and biases, and 32-bit accumulators.

Remember that a `forgiving factor` forgives a drop in a metric such as `accuracy` if the gains of the model are much bigger than the drop. For example, it corresponds to the sentence *we allow $\tt{delta}\%$ reduction in accuracy if the quantized model has $\tt{rate} \times$ smaller energy than the original model*, being a multiplicative factor to the metric. It is computed by $1 + \tt{delta} \times  \log_{\tt{rate}}(\tt{stress} \times \tt{reference\_cost} / \tt{trial\_cost})$.

In [15]:
goal = {
    "type": "energy",
    "params": {
        "delta_p": 8.0,
        "delta_n": 8.0,
        "rate": 2.0,
        "stress": 1.0,
        "process": "horowitz",
        "parameters_on_memory": ["sram", "sram"],
        "activations_on_memory": ["sram", "sram"],
        "rd_wr_on_io": [False, False],
        "min_sram_size": [0, 0],
        "source_quantizers": ["int8"],
        "reference_internal": "int8",
        "reference_accumulator": "int32"
        }
}

There are a few more things we need to define. Let's bundle them on a dictionary and pass them to `AutoQKeras`.  We will try a maximum of 10 trials (`max_trials`) just to limit the time we will spend finding the best quantization here.  Please note that this parameter is not valid if you are running in `hyperband` mode.

`output_dir` is the directory where we will store our results. Since we are running on a colab, we will let `tempfile` chooce a directory for us.

`learning_rate_optimizer` allows `AutoQKeras` to change the optimization function and the `learning_rate` to try to improve the quantization results. Since it is still experimental, it may be the case that in some cases it will get worse results. 

Because we are tuning filters as well, we should set `transfer_weights` to `False` as the trainable parameters will have different shapes.

In `AutoQKeras` we have three modes of operation: `random`, `bayesian` and `hyperband`. I recommend the user to refer to `KerasTuner` (https://keras-team.github.io/keras-tuner/) for a complete description of them.

`tune_filters` can be set to `layer`, `block` or `none`. If `tune_filters` is `block`, we change the filters by the same amount for all layers being quantized in the trial. If `tune_filters` is `layer`, we will possibly change the number of filters for each layer independently. Finally, if `tune_filters` is `none`, we will not perform filter tuning.

Together with `tune_filters`, `tune_filter_exceptions` allows the user to specify by a regular expression which filters we should not perform filter tuning, which is especially good for the last layers of the network.

Filter tuning is a very important feature of `AutoQKeras`. When we deep quantize a model, we may need less or more filters for each layer (and you can guess we do not know a priori how many filters we will need for each layer). Let me give you a rationale behind this.

- **less filters**: let us assume we have two set of filter coefficients we want quantize: $[-0.3, 0.2, 0.5, 0.15]$ and $[-0.5, 0.4, 0.1, 0.65]$. If we apply a $\tt{binary}$ quantizer with $\tt{scale} = \big\lceil \log_2(\frac{\sum |w|}{N}) \big\rceil$, where $w$ are the filter coefficients and $N$ is the number of coefficients, we will end up with the same filter $\tt{binary}([-0.3, 0.2, 0.5, 0.15]) = \tt{binary}([-0.5, 0.4, 0.1, 0.65]) = [-1,1,1,1] \times 0.5$. In this case we are assuming the $\tt{scale}$ is a power-of-2 number so that it can be efficiently implemented by a shift operation;

- **more filters**: it is clear that quantization will drop information (just look at the example above) and deep quantization will drop more information, so to recover some of the boundary regions in layers that perform feature extraction, we may need to add more filters to the layer when we quantize it.

We do not want to quantize the `softmax` layer, which is the last layer of the network. In `AutoQKeras`, you can specify the indexes that you want to perform quantization by specifying the corresponding index of the layer in `Keras`, i.e. if you can get the layer as `model.layers[i]` in `Keras`, `i` is the index of the layer.

Finally, for data parallel distributed training, we should pass the strategy in `distribution_strategy` to `KerasTuner`.

In [16]:
run_config = {
  "output_dir": tempfile.mkdtemp(),
  "goal": goal,
  "quantization_config": quantization_config,
  "learning_rate_optimizer": False,
  "transfer_weights": False,
  "mode": "random",
  "seed": 42,
  "limit": limit,
  "tune_filters": "layer",
  "tune_filters_exceptions": "^dense",
  "distribution_strategy": cur_strategy,
  # first layer is input, layer two layers are softmax and flatten
  "layer_indexes": range(1, len(model.layers) - 1),
  "max_trials": 20
}

print("quantizing layers:", [model.layers[i].name for i in run_config["layer_indexes"]])

quantizing layers: ['conv2d_0', 'bn_0', 'act_0', 'drop_0', 'conv2d_1', 'bn_1', 'act_1', 'drop_1', 'conv2d_2', 'bn_2', 'act_2', 'drop_2', 'conv2d_3', 'bn_3', 'act_3', 'drop_3', 'conv2d_4', 'bn_4', 'act_4', 'drop_4', 'flatten', 'dense']


In [17]:
autoqk = AutoQKeras(model, metrics=["acc"], custom_objects=custom_objects, **run_config)
autoqk.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=1024, epochs=20)

Trial 20 Complete [00h 01m 43s]
val_score: 1.355238437652588

Best val_score So Far: 1.3851507902145386
Total elapsed time: 00h 45m 09s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Now, let's see which model is the best model we got.


In [18]:
qmodel = autoqk.get_best_model()
qmodel.save_weights("qmodel.h5")

learning_rate: 0.019999999552965164
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_0 (QConv2D)          (None, 14, 14, 24)        216       
                                                                 
 bn_0 (QBatchNormalization)  (None, 14, 14, 24)        96        
                                                                 
 act_0 (QActivation)         (None, 14, 14, 24)        0         
                                                                 
 drop_0 (Dropout)            (None, 14, 14, 24)        0         
                                                                 
 conv2d_1 (QConv2D)          (None, 7, 7, 16)          3456      
                                                                 
 bn_1 (QBatchNormalizatio

We got here >90% reduction in energy when compared to 8-bit tensors and 32-bit accumulators. Remember that our original number was 3.3 uJ for fp32.  The end model has 11 nJ for the quantized model as opposed to 204 nJ for the 8-bit original quantized model. As these energy numbers are from high-level energy models, you should remember to consider the relations between them, and not the actual numbers.

Let's train this model to see how much accuracy we can get of it.

In [19]:
qmodel.load_weights("qmodel.h5")
with cur_strategy.scope():
  optimizer = Adam(lr=0.02)
  qmodel.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["acc"])
  qmodel.fit(x_train, y_train, epochs=200, batch_size=4096, validation_data=(x_test, y_test))

Epoch 1/200
15/15 [==============================] - 14s 486ms/step - loss: 1.2808 - acc: 0.5802 - val_loss: 1.3874 - val_acc: 0.7358
Epoch 2/200
15/15 [==============================] - 5s 342ms/step - loss: 0.5959 - acc: 0.8041 - val_loss: 0.8395 - val_acc: 0.8385
Epoch 3/200
15/15 [==============================] - 5s 357ms/step - loss: 0.4537 - acc: 0.8545 - val_loss: 0.4389 - val_acc: 0.8949
Epoch 4/200
15/15 [==============================] - 5s 317ms/step - loss: 0.3832 - acc: 0.8787 - val_loss: 0.3548 - val_acc: 0.9037
Epoch 5/200
15/15 [==============================] - 5s 326ms/step - loss: 0.3353 - acc: 0.8931 - val_loss: 0.2432 - val_acc: 0.9340
Epoch 6/200
15/15 [==============================] - 5s 332ms/step - loss: 0.3011 - acc: 0.9056 - val_loss: 0.1975 - val_acc: 0.9420
Epoch 7/200
15/15 [==============================] - 5s 314ms/step - loss: 0.2697 - acc: 0.9151 - val_loss: 0.1561 - val_acc: 0.9536
Epoch 8/200
15/15 [==============================] - 5s 336ms/step -

One of problems of trying to quantize the whole thing in one shot is that we may end up with too many choices to make, which will make the entire search space very high. In order to reduce the search space, `AutoQKeras` has two methods to enable users to cope with the explosion of choices.

## Grouping Layers to Use the Same Choice

In this case, we can provide regular expressions to `limit` to specify layer names that should be grouped together. In our example, suppose we want to group  convolution layers (except the first one) and all activations except the last one to use the same quantization.

For the first convolution layer, we want to limit the quantization types to fewer choices as the input is already an 8-bit number.  The last activation will be fed to a feature classifier layer, so we may leave it with more bits. Because our `dense` is actually a `Conv2D` operation, we will enable 8-bits for the weights by layer name. 

We first need to look at the names of the layers for this. 

In [20]:
pprint.pprint([layer.name for layer in model.layers])

['input',
 'conv2d_0',
 'bn_0',
 'act_0',
 'drop_0',
 'conv2d_1',
 'bn_1',
 'act_1',
 'drop_1',
 'conv2d_2',
 'bn_2',
 'act_2',
 'drop_2',
 'conv2d_3',
 'bn_3',
 'act_3',
 'drop_3',
 'conv2d_4',
 'bn_4',
 'act_4',
 'drop_4',
 'flatten',
 'dense',
 'softmax']


Convolution layers for `mnist` have names specified as `conv2d_[01234]`. Activation layers have names specified as `act_[01234]`. So, we can create the following regular expressions to reduce the search space in our model.

Please note that layer class names always select different quantizers, so the user needs to specify a pattern for layer names if he/she wants to use the same quantization for the group of layers.

You can see here another feature of the limit. You can specify the maximum number of bits, or cherry pick which quantizers you want to try for a specific layer if instead of the maximum number of bits you specify a list of quantizers fron `quantization_config`.

In [21]:
limit = {
    "Dense": [8, 8, 4],
    "Conv2D": [4, 8, 4],
    "DepthwiseConv2D": [4, 8, 4],
    "Activation": [4],
    "BatchNormalization": [],

    "^conv2d_0$": [
                   ["binary", "ternary", "quantized_bits(2,1,1,alpha=1.0)"],
                   8, 4
    ],
    "^conv2d_[1234]$": [4, 8, 4],
    "^act_[0123]$": [4],
    "^act_4$": [8],
    "^dense$": [8, 8, 4]
}

In [22]:
run_config = {
  "output_dir": tempfile.mkdtemp(),
  "goal": goal,
  "quantization_config": quantization_config,
  "learning_rate_optimizer": False,
  "transfer_weights": False,
  "mode": "random",
  "seed": 42,
  "limit": limit,
  "tune_filters": "layer",
  "tune_filters_exceptions": "^dense",
  "distribution_strategy": cur_strategy,
  "layer_indexes": range(1, len(model.layers) - 1),
  "max_trials": 40
}

In [23]:
autoqk = AutoQKeras(model, metrics=["acc"], custom_objects=custom_objects, **run_config)
autoqk.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=1024, epochs=20)

Trial 40 Complete [00h 02m 41s]
val_score: 1.2728118896484375

Best val_score So Far: 1.4286894798278809
Total elapsed time: 02h 01m 04s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Let's see the reduction now.

In [24]:
qmodel = autoqk.get_best_model()
qmodel.save_weights("qmodel.h5")

learning_rate: 0.019999999552965164
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_0 (QConv2D)          (None, 14, 14, 8)         72        
                                                                 
 bn_0 (QBatchNormalization)  (None, 14, 14, 8)         32        
                                                                 
 act_0 (QActivation)         (None, 14, 14, 8)         0         
                                                                 
 drop_0 (Dropout)            (None, 14, 14, 8)         0         
                                                                 
 conv2d_1 (QConv2D)          (None, 7, 7, 16)          1152      
                                                                 
 bn_1 (QBatchNormalizatio

Let's train this model for more time to see how much we can get in accuracy.

In [25]:
qmodel.load_weights("qmodel.h5")
with cur_strategy.scope():
  optimizer = Adam(lr=0.02)
  qmodel.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["acc"])
  qmodel.fit(x_train, y_train, epochs=200, batch_size=4096, validation_data=(x_test, y_test))

Epoch 1/200
15/15 [==============================] - 16s 455ms/step - loss: 1.3290 - acc: 0.5535 - val_loss: 2.1514 - val_acc: 0.6137
Epoch 2/200
15/15 [==============================] - 3s 207ms/step - loss: 0.6596 - acc: 0.7860 - val_loss: 1.5057 - val_acc: 0.7240
Epoch 3/200
15/15 [==============================] - 4s 247ms/step - loss: 0.4349 - acc: 0.8612 - val_loss: 1.3564 - val_acc: 0.7383
Epoch 4/200
15/15 [==============================] - 3s 227ms/step - loss: 0.3452 - acc: 0.8933 - val_loss: 0.5676 - val_acc: 0.8611
Epoch 5/200
15/15 [==============================] - 4s 237ms/step - loss: 0.2982 - acc: 0.9086 - val_loss: 0.3139 - val_acc: 0.9130
Epoch 6/200
15/15 [==============================] - 4s 253ms/step - loss: 0.2655 - acc: 0.9182 - val_loss: 0.1886 - val_acc: 0.9448
Epoch 7/200
15/15 [==============================] - 3s 225ms/step - loss: 0.2581 - acc: 0.9219 - val_loss: 0.1904 - val_acc: 0.9438
Epoch 8/200
15/15 [==============================] - 4s 239ms/step -

## Quantization by Blocks

In the previous section, we enforced that all decisions were the same in order to reduce the number of options to quantize a model. 

Another approach is still to allow models to have each block of layers to makde their own choice, but quantizing the blocks sequentially, either from inputs to outputs, or by quantizing higher energy blocks first.

The rationale for this method is that if we quantize the blocks one by one, and assuming that each block has $N$ choices, and $B$ blocks, we end up trying $N B$ options, instead of $N^B$ choices.  The reader should note that this is an approximation as there is no guarantee that we will obtain the best quantization possible.

Should you do sequential from inputs to outputs or starting from the block that has the highest impact?

If you have a network like ResNet, and if you want to do filter tuning, you need to block the layers by the resnet definition of a block, i.e. including full identity or convolutional blocks, and quantize the model from inputs to outputs, so that you can preserve at each stage the number of channels for the residual block. 

In order to perform quantization by blocks, you need to specify two other parameters in our `run_config`. `blocks` is a list of regular expressions of the groups you want to quantize. If a layer does not match the block pattern, it will not be quantized.  `schedule_block` specifies the mode for block quantization scheduling. It can be `sequential` or `cost` if you want to schedule first the blocks by decreasing cost size (energy or bits).

In this model, there are a few optimizations that we perform automatically. First, we dynamically reduce the learning rate of the blocks that we have already quantized as setting them to not-trainable does not seem to work, so we still allow them to train, but at a slower pace. In addition, we try to dynamically adjust the learning rate for the layer we are trying to quantize as opposed to the learning rate of the unquantized layers. Finally, we transfer the weights of the models we have already quantized whenever we can do (if the shapes remain the same). 

Regardless on how we schedule the operations, we amortize the nubmer of trials for the cost of the block (energy or bits with respect to the total energy or number of bits of the network).

Instead of invoking `AutoQKeras` now, we will invoke `AutoQKeras` scheduler.

In [26]:
run_config = {
  "output_dir": tempfile.mkdtemp(),
  "goal": goal,
  "quantization_config": quantization_config,
  "learning_rate_optimizer": False,
  "transfer_weights": False,
  "mode": "random",
  "seed": 42,
  "limit": limit,
  "tune_filters": "layer",
  "tune_filters_exceptions": "^dense",
  "distribution_strategy": cur_strategy,
  "layer_indexes": range(1, len(model.layers) - 1),
  "max_trials": 40,

  "blocks": [
    "^.*_0$",
    "^.*_1$",
    "^.*_2$",
    "^.*_3$",
    "^.*_4$",
    "^dense"
  ],
  "schedule_block": "cost"
}

Because specifying regular expressions is error prone, we recommend that you first try to run `AutoQKerasScheduler` in debug mode to print the blocks.

In [27]:
pprint.pprint([layer.name for layer in model.layers])
autoqk = AutoQKerasScheduler(model, metrics=["acc"], custom_objects=custom_objects, debug=True, **run_config)
autoqk.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=1024, epochs=20)

['input',
 'conv2d_0',
 'bn_0',
 'act_0',
 'drop_0',
 'conv2d_1',
 'bn_1',
 'act_1',
 'drop_1',
 'conv2d_2',
 'bn_2',
 'act_2',
 'drop_2',
 'conv2d_3',
 'bn_3',
 'act_3',
 'drop_3',
 'conv2d_4',
 'bn_4',
 'act_4',
 'drop_4',
 'flatten',
 'dense',
 'softmax']
... block cost: 199940 / 574175
... adjusting max_trials for this block to 13
Pattern 0 is : {'bn_1': [], '^(conv2d_1)$': [4, 8, 4], '^(act_1)$': [4]}
... block cost: 195890 / 574175
... adjusting max_trials for this block to 13
Pattern 1 is : {'bn_2': [], '^(conv2d_2)$': [4, 8, 4], '^(act_2)$': [4]}
... block cost: 98754 / 574175
... adjusting max_trials for this block to 10
Pattern 2 is : {'bn_3': [], '^(conv2d_3)$': [4, 8, 4], '^(act_3)$': [4]}
... block cost: 69621 / 574175
... adjusting max_trials for this block to 10
Pattern 3 is : {'bn_4': [], '^(conv2d_4)$': [4, 8, 4], '^(act_4)$': [8]}
... block cost: 8788 / 574175
... adjusting max_trials for this block to 10
Pattern 4 is : {'bn_0': [], '^(conv2d_0)$': [['binary', 'ternar

All blocks seem to be fine. Let's find the best quantization now.

In [28]:
autoqk = AutoQKerasScheduler(model, metrics=["acc"], custom_objects=custom_objects, **run_config)
autoqk.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=1024, epochs=20)

Trial 10 Complete [00h 00m 50s]
val_score: 1.4008370637893677

Best val_score So Far: 1.416741967201233
Total elapsed time: 00h 07m 44s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in /tmp/tmp7td4spyh_1/5
Showing 10 best trials
Trial summary
Hyperparameters:
^(dense)$_kernel_quantizer: quantized_bits(8,0,1,alpha=1.0)
^(dense)$_bias_quantizer: quantized_bits(8,3,1)
Score: 1.416741967201233
Trial summary
Hyperparameters:
^(dense)$_kernel_quantizer: quantized_bits(4,0,1,alpha=1.0)
^(dense)$_bias_quantizer: quantized_bits(4,0,1)
Score: 1.4149357080459595
Trial summary
Hyperparameters:
^(dense)$_kernel_quantizer: stochastic_binary
^(dense)$_bias_quantizer: quantized_bits(8,3,1)
Score: 1.414434552192688
Trial summary
Hyperparameters:
^(dense)$_kernel_quantizer: binary
^(dense)$_bias_quantizer: quantized_bits(4,0,1)
Score: 1.4135743379592896
Trial summary
Hyperparameters:
^(dense)$_kernel_quantizer: stochastic_binary
^(dense)$_bias_quantizer: quantized_bits(4,0,1)
Score: 1.4087834358215332
Trial summary
Hyperparameters:
^(dense)$_kernel_quantizer: ternary
^(dense)$_bias_quantizer: quantized_bits(4,0,1)
Score: 1.4080085754394531
Trial summary
Hype

INFO:tensorflow:Assets written to: /tmp/tmp7td4spyh_1/model_block_5/assets


INFO:tensorflow:Assets written to: /tmp/tmp7td4spyh_1/model_block_5/assets


In [29]:
qmodel = autoqk.get_best_model()
qmodel.save_weights("qmodel.h5")

stats: delta_p=0.08 delta_n=0.08 rate=2.0 trial_size=11506 reference_size=574175
       delta=45.13%
Total Cost Reduction:
       11506 vs 574175 (-98.00%)
conv2d_0             f=8 binary(alpha='auto_po2') 
bn_0                 QBN, mean=[0. 0. 0. 0. 0. 0. 0. 0.]
act_0                quantized_relu(4,2)
conv2d_1             f=16 binary(alpha='auto_po2') 
bn_1                 QBN, mean=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
act_1                quantized_relu(3,1)
conv2d_2             f=24 quantized_bits(4,0,1,alpha=1.0) 
bn_2                 QBN, mean=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
act_2                quantized_relu_po2(4,4)
conv2d_3             f=32 quantized_bits(4,0,1,alpha=1.0) 
bn_3                 QBN, mean=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
act_3                quantized_relu(3,1)
conv2d_4             f=64 ternary(alpha='auto_po2') 
bn_4                 QBN, mea

In [30]:
qmodel.load_weights("qmodel.h5")
with cur_strategy.scope():
  optimizer = Adam(lr=0.02)
  qmodel.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["acc"])
  qmodel.fit(x_train, y_train, epochs=200, batch_size=4096, validation_data=(x_test, y_test))

Epoch 1/200
15/15 [==============================] - 5s 158ms/step - loss: 1.3593 - acc: 0.5442 - val_loss: 2.8060 - val_acc: 0.4612
Epoch 2/200
15/15 [==============================] - 2s 110ms/step - loss: 0.6045 - acc: 0.8052 - val_loss: 2.0059 - val_acc: 0.6075
Epoch 3/200
15/15 [==============================] - 2s 107ms/step - loss: 0.4102 - acc: 0.8723 - val_loss: 1.0139 - val_acc: 0.7363
Epoch 4/200
15/15 [==============================] - 2s 110ms/step - loss: 0.3345 - acc: 0.8951 - val_loss: 0.3237 - val_acc: 0.9072
Epoch 5/200
15/15 [==============================] - 2s 103ms/step - loss: 0.2875 - acc: 0.9101 - val_loss: 0.1854 - val_acc: 0.9455
Epoch 6/200
15/15 [==============================] - 2s 104ms/step - loss: 0.2643 - acc: 0.9179 - val_loss: 0.1530 - val_acc: 0.9547
Epoch 7/200
15/15 [==============================] - 2s 105ms/step - loss: 0.2408 - acc: 0.9247 - val_loss: 0.1298 - val_acc: 0.9598
Epoch 8/200
15/15 [==============================] - 2s 103ms/step - 

Perfect! You have learned how to perform automatic quantization using AutoQKeras with QKeras.